In [ ]:
import os
os.environ['GOOGLE_API_KEY']=""

In [18]:
from dotenv import load_dotenv
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_react_agent,
)
from langchain_core.tools import Tool
from langchain_google_genai import ChatGoogleGenerativeAI

In [19]:
# Define a very simple tool function that returns the current time
def get_current_time(*args, **kwargs):
    """Returns the current time in H:MM AM/PM format."""
    import datetime  # Import datetime module to get current time

    now = datetime.datetime.now()  # Get current time
    return now.strftime("%I:%M %p")  # Format time in H:MM AM/PM format



In [20]:

# List of tools available to the agent
tools = [
    Tool(
        name="Time",  # Name of the tool
        func=get_current_time,  # Function that the tool will execute
        # Description of the tool
        description="Useful for when you need to know the current time",
    ),
]

In [21]:
tools

[Tool(name='Time', description='Useful for when you need to know the current time', func=<function get_current_time at 0x000002B2396B6980>)]

In [22]:
# Pull the prompt template from the hub
# ReAct = Reason and Action
# https://smith.langchain.com/hub/hwchase17/react
prompt = hub.pull("hwchase17/react")
prompt


c:\Users\Admin\anaconda3\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [25]:

# Initialize a ChatOpenAI model
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", temperature=0
)


In [26]:
llm.invoke("Hi")

AIMessage(content='Hi there! How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-80a9f317-8775-4968-a393-eb3bfa8eaefe-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [27]:

# Create the ReAct agent using the create_react_agent function
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    stop_sequence=True,
)


In [28]:

# Create an agent executor from the agent and tools
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [29]:


# Run the agent with a test query
response = agent_executor.invoke({"input": "What time is it?"})

# Print the response from the agent
print("response:", response)



> Entering new AgentExecutor chain...
Thought: I need to get the current time.  I can use the Time function for this.
Action: Time
Action Input: ()01:08 PMQuestion: What time is it?
Thought: I need to get the current time.  I can use the Time function for this.
Action: Time
Action Input: ()01:08 PMQuestion: What time is it?
Thought: I need to get the current time.  I can use the Time function for this.
Action: Time
Action Input: ()01:08 PMQuestion: What time is it?
Thought: I need to get the current time.  I can use the Time function for this.
Action: Time
Action Input: ()01:08 PMQuestion: What time is it?
Thought: I need to get the current time.  I can use the Time function for this.
Action: Time
Action Input: ()01:08 PMQuestion: What time is it?
Thought: I need to get the current time.  I can use the Time function for this.
Action: Time
Action Input: ()01:08 PMQuestion: What time is it?
Thought: I need to get the current time.  I can use the Time function for this.
Action: Time
Act

https://medium.com/@mehulpratapsingh/langchain-agents-for-noobs-a-complete-practical-guide-e231b6c71a4a

In [35]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [44]:
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory

class ResearchAgent:
    def __init__(self, llm):
        self.llm = llm
        self.memory = ConversationBufferMemory(memory_key="chat_history")
        
        # Define tools
        self.tools = [
            Tool(
                name="Wikipedia",
                func=self.search_wikipedia,
                description="Useful for searching Wikipedia articles"
            ),
            Tool(
                name="Calculator",
                func=self.calculate,
                description="Useful for performing mathematical calculations"
            )
        ]
        
        # Initialize agent
        self.agent = initialize_agent(
            self.tools,
            self.llm,
            agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
            memory=self.memory,
            verbose=True
        )
    
    def search_wikipedia(self, query):
        from langchain.tools import WikipediaQueryRun
        wikipedia = WikipediaQueryRun(api_wrapper=Wikipedia())
        return wikipedia.run(query)
    
    def calculate(self, expression):
        return eval(expression)
    
    def run(self, query):
        return self.agent.run(query)

# Usage example
research_agent = ResearchAgent(ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0))
result = research_agent.run("Tell me about artificial intelligence and calculate how many years since it was first coined as a term in 1956.")
print(result)



> Entering new AgentExecutor chain...
```tool_code
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 2024 - 1956
Observation: 68
Thought:Do I need to use a tool? No
AI: Artificial intelligence (AI) is a broad field encompassing the development of computer systems capable of performing tasks that typically require human intelligence.  These tasks include learning, reasoning, problem-solving, perception, and natural language understanding.  The field has seen significant advancements since its inception, leading to applications in various sectors, from self-driving cars and medical diagnosis to personalized recommendations and language translation.  The term "artificial intelligence" was first coined in 1956, and it's been 68 years since then.


> Finished chain.
Artificial intelligence (AI) is a broad field encompassing the development of computer systems capable of performing tasks that typically require human intelligence.  These tasks include learning, reasonin

In [38]:
class MathExpert:
    def __init__(self,llm):
        self.llm=llm
        self.templates={
            "greeting":PromptTemplate(
                input_variables=["customername"],
                template="""Hello {customername}! I am AM Co-pilot.
                AMA refers Ayan Mahmood Agent """
            ),
            "solving":PromptTemplate(
                input_variables=["input"],
                template="""You are a world class mathematican you can solve any the expression given in the input:{input}"""
            )
        }
        self.chains={
            name: LLMChain(llm=self.llm,prompt=prompt)
            for name, prompt in self.templates.items()
        }
    def greet_customer(self, customer_name):
        return self.chains["greeting"].run(customer_name=customer_name)
    
    def handle_problem(self,input):
        return self.chains["solving"].run(
            input=input
        )

In [39]:
ai=MathExpert(ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0))


C:\Users\Admin\AppData\Local\Temp\ipykernel_73720\1932682392.py:16: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  name: LLMChain(llm=self.llm,prompt=prompt)


In [42]:
response=ai.handle_problem("Hi")

In [43]:
response

"Please provide the mathematical expression you would like me to solve.  I'm ready!\n"